<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-2/blob/main/CS6910_Assignment2_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS6910-Assignment2-Part B
Using Pretrained model for Image Classification

In [6]:
!pip install -q wandb
import wandb
from wandb.keras import WandbCallback

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [7]:
def get_data():
    #path=r"C:\Users\vamsi_oe20s302\Downloads\nature_12K\inaturalist_12K\train"
    path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"

    # Training dataset:
    train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
        directory=path,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(256,256),
        shuffle=True,
        seed=19,
        validation_split=0.1,
        subset='training'
    )
    # Validation dataset:
    valid_dataset=tf.keras.preprocessing.image_dataset_from_directory(
        directory=path,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(256,256),
        shuffle=True,
        seed=19,
        validation_split=0.1,
        subset='validation'
    )
    
    #return train_data, valid_data
    return train_dataset, valid_dataset


In [8]:
def get_augmented_data():
    #path=r"C:\Users\vamsi_oe20s302\Downloads\nature_12K\inaturalist_12K\train"
    path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"
    training_data_augmentation=ImageDataGenerator(rescale=1./255,
                                        height_shift_range=0.2,
                                        width_shift_range=0.2,
                                        horizontal_flip=True,
                                        zoom_range=0.2,
                                        fill_mode="nearest",
                                        validation_split = 0.1)

    # Validation data is not being augmented
    validation_data_augmentation=ImageDataGenerator(
        validation_split=0.1
    )

    train_aug=training_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='training')
    valid_aug=validation_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='validation')

    return train_aug, valid_aug


#Transfer Learning 
1. Reference 1: https://www.tensorflow.org/tutorials/images/transfer_learning
2. Reference 2: https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751

In [9]:
class build_cnn(Model):
  
  def __init__(self,model_name,tune=False,freeze=20):
    
    super(build_cnn,self).__init__()
    
    self.image_shape=(256,256,3)
    self.model_name=model_name
    self.initialize_model()

    if tune == True:
      self.model.trainable= True
      train_and_tune=len(self.model.layers)-freeze

      for layer in self.model.layers[:freeze]:
        layer.trainable = False
    else:
      self.model.trainable = False

  # here we are using global average pooling layer as classifier on top of 
  # the pretrained convolution layers 
    self.avg_pool=layers.GlobalAveragePooling2D()
    self.dense=layers.Dense(10)
  
  # Initializing the model
  # NOTE: By using include_top=False, we are not using the final classification
  # layer of the pre trained model.
  def inceptionv3(self):
      model=tf.keras.applications.InceptionV3(input_shape=self.image_shape,
                                            include_top=False,
                                            weights='imagenet')
      return model

  def resnet50(self):
      model=tf.keras.applications.ResNet50(input_shape=self.image_shape,
                                         include_top=False,
                                         weights='imagenet')
      return model

  def inceptionresnetv2(self):
      model=tf.keras.applications.InceptionResNetV2(input_shape=self.image_shape,
                                                        include_top=False,
                                                        weights='imagenet')
      return model
  
  def xception(self):
      model=tf.keras.applications.Xception(input_shape=self.image_shape,
                                         include_top=False,
                                         weights='imagenet')
      return model
    
  
  def initialize_model(self):
    
    if self.model_name  == "InceptionV3":
      self.model=self.inceptionv3()
    
    elif self.model_name == "InceptionResNetV2":
      self.model=self.inceptionresnetv2()
    
    elif self.model_name == "ResNet50":
      self.model=self.resnet50()

    elif self.model_name == "Xception":
      self.model=self.xception()
    
    else:
      print("Enter a valid model name")
  
  
  def call(self,inputs):
    a=self.model(inputs)
    a=self.avg_pool(a)
    a=self.dense(a)
    return a

In [10]:
# base_model = "ResNet50"
# tune = False
# freeze = 20

# model = build_cnn(base_model, tune, freeze)

# base_learning_rate = 0.0001
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
#                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
#                 metrics=['accuracy'])

# train_aug_gen=get_augmented_data()[0]
# val_aug_gen=get_augmented_data()[1]

# history = model.fit(train_aug_gen, steps_per_epoch=62, \
#                     epochs=10, validation_data=val_aug_gen, \
#                     validation_steps=62, workers=8)

# Wandb Sweeping

In [3]:
hyperparameters={
    'model_name':{'values': ["InceptionV3","InceptionResNetV2","ResNet50","Xception"]},
    'tune':{'values':[False, True]},
    'data_augmentation':{'values':[False,True]}
}
sweep_config = {
      'method' : 'bayes',
      'metric' :{
          'name': 'val_acc',
          'goal': 'maximize'
      },
      'parameters': hyperparameters
    }

In [17]:
def wandb_train(config=sweep_config):
  # initializing wandb with the above configuration
  wandb.init(config=config)
  # collecting the configuration information
  config=wandb.init().config
  # setting up the name for each run example : Model_Xception_tune_False_aug_True
  run_name='Model_{}_tune_{}_Data_aug_{}'.format(config.model_name,config.tune,config.data_augmentation)
  wandb.run.name=run_name
  wandb_log=True
  # caling the model 
  model=build_cnn(config.model_name,config.tune)
  
  learning_rate=0.0001
  
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  # Getting data 
  train_data,val_data=get_data()
  train_aug_data,val_aug_data=get_augmented_data()

  if config.data_augmentation == False:
    history = model.fit(train_data, epochs=10, validation_data=val_data,
                        callbacks = [wandb.keras.WandbCallback()])
  else:
    history = model.fit(train_aug_data, steps_per_epoch=62, epochs=10,
                        validation_data=val_aug_data, validation_steps=62,
                        workers=10, callbacks = [wandb.keras.WandbCallback()])  
  
  wandb.run.finish()

In [19]:
!wandb login --relogin
entity_name = "safi-vamsi-cs6910"
project_name = "Assignment 2"

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_id=wandb.sweep(sweep_config,entity=entity_name,project=project_name)
wandb.agent(sweep_id,wandb_train)

Create sweep with ID: lkwg7x5q
Sweep URL: https://wandb.ai/safi-vamsi-cs6910/Assignment%202/sweeps/lkwg7x5q


wandb: Agent Starting Run: csq8ek2o with config:
wandb: 	data_augmentation: True
wandb: 	model_name: Xception
wandb: 	tune: False


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Found 9999 files belonging to 10 classes.
Using 9000 files for training.
Found 9999 files belonging to 10 classes.
Using 999 files for validation.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
62/62 [==============================] - ETA: 0s - loss: 2.2772 - accuracy: 0.1613

wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


...

accuracy,▁▃▅▇▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▅▄▃▃▂▂▁▁
val_accuracy,▁▃▄▃▃▅▃▄▃█
val_loss,▁▃▁▂▃▅▅▆▇█
accuracy,0.69859
best_epoch,2
best_val_loss,16.24827
epoch,9
loss,1.18472
val_accuracy,0.11485


wandb: Agent Starting Run: 8i4e9o8h with config:
wandb: 	data_augmentation: True
wandb: 	model_name: Xception
wandb: 	tune: False


Found 9999 files belonging to 10 classes.
Using 9000 files for training.
Found 9999 files belonging to 10 classes.
Using 999 files for validation.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
62/62 [==============================] - ETA: 0s - loss: 2.2263 - accuracy: 0.1996

wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


62/62 [==============================] - 95s 1s/step - loss: 2.2263 - accuracy: 0.1996 - val_loss: 13.0990 - val_accuracy: 0.0730 - _timestamp: 1648463335.0000 - _runtime: 104.0000
Epoch 2/10
62/62 [==============================] - ETA: 0s - loss: 1.9941 - accuracy: 0.3985